### Configuración inicial

In [2]:
# Librerias y Variables de enter

# import sys
# sys.path.append(r"C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages")
import requests # peticiones HTML
import pandas as pd # manejo de datos
import matplotlib.pyplot as plt # graficacion
import pytz
from datetime import datetime, timedelta

# Variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables desde el archivo .env
load_dotenv()

# Obtener las variables desde el entorno
account_name = os.getenv("VTEX_ACCOUNT_NAME")
app_key = os.getenv("VTEX_APP_KEY")
app_token = os.getenv("VTEX_APP_TOKEN")

# Validar que las variables se cargaron correctamente
if not all([account_name, app_key, app_token]):
    raise ValueError("Faltan credenciales en el archivo .env")

In [3]:
# Configuración de la URL API y headers
base_url = f"https://{account_name}.vtexcommercestable.com.br/api/oms/pvt/orders"
headers = {
    "X-VTEX-API-AppKey": app_key,
    "X-VTEX-API-AppToken": app_token,
    "Content-Type": "application/json",
}

### Llamadas API

In [11]:
# Función para convertir fechas en formato DD/MM/YYYY al formato requerido por VTEX
def convert_to_vtex_datetime(date):
  if isinstance(date, str):  # Convertir string a datetime si es necesario
      local_time = datetime.strptime(date, "%d/%m/%Y")
  else:
      local_time = date  # Si ya es datetime, usarlo directamente

  start_utc = local_time + timedelta(hours=3)  # Convertir a UTC (sumar 3 horas)
  start_str = start_utc.strftime("%Y-%m-%dT%H:%M:%S.000Z")
  end_utc = start_utc + timedelta(hours=23, minutes=59, seconds=59, milliseconds=999)
  end_str = end_utc.strftime("%Y-%m-%dT%H:%M:%S.999Z")
  return start_str, end_str

In [16]:
# Función llamada a la API para obtener pedidos de un día
def get_orders_for_day(date):
  # Convertir la fecha al formato VTEX
  start_date, end_date = convert_to_vtex_datetime(date)

  # Parámetros para filtrar órdenes
  params = {
    'orderBy': 'sequence',
    'f_creationDate': f'creationDate:[{start_date} TO {end_date}]',  # Filtro de fecha
    "page": 1,  # de 1 a 30
    "per_page": 100,  # de 1 a 100
  }

  orders = []
  page = 1
  while True:
    params['page'] = page
    response = requests.get(base_url, headers=headers, params=params)

    # Verificar si la respuesta es válida
    try:
      data = response.json()
    except ValueError as e:
      print(f"Error al procesar la respuesta JSON: {response.text}")
      break

    # Asegurarse de que "list" exista en la respuesta
    if "list" not in data:
      print(f"Error: 'list' no encontrado en la respuesta: {data}")
      break

    orders_list = data["list"]
    if not orders_list:
      break

    for order in orders_list:
      order_data = {
        'sequence': order.get('sequence', ''),
        'creationDate': order.get('creationDate', ''),
        'totalItems': order.get('totalItems', 0),
        'totalValue': order.get('value', 0),
        'paymentNames': ', '.join([payment.get('paymentSystem', '') for payment in order.get('payments', [])]),
        'statusDescription': order.get('statusDescription', ''),
        'hostname': order.get('hostname', ''),
        'origin': order.get('origin', '')
      }
      orders.append(order_data)

    page += 1

  return orders

In [14]:
# Función para iterar por días en un rango y obtener todos los pedidos
def get_orders_by_date_range(start_date_input, end_date_input):
  start_date = datetime.strptime(start_date_input, "%d/%m/%Y")
  end_date = datetime.strptime(end_date_input, "%d/%m/%Y")

  all_orders = []

  current_date = start_date
  while current_date <= end_date:
      print(f"Obteniendo pedidos para el día: {current_date.strftime('%d/%m/%Y')}")
      daily_orders = get_orders_for_day(current_date)
      all_orders.extend(daily_orders)
      current_date += timedelta(days=1)

  return all_orders

In [17]:
# Obtener los pedidos de 1 PERIODO

# Fechas para filtrar (en formato DD/MM/YYYY)
fecha_inicio = "30/12/2024"
fecha_fin = "31/12/2024"

# Obtener los pedidos
orders = get_orders_by_date_range(fecha_inicio, fecha_fin)

# Convertir los pedidos a un DataFrame de pandas
df_orders = pd.DataFrame(orders)

# Guardar el DataFrame a un archivo CSV
df_orders.to_csv('pedidos_2024.csv', index=False)

print("Datos de pedidos guardados en 'pedidos_2024.csv'")

Obteniendo pedidos para el día: 30/12/2024


NameError: name 'date_str' is not defined